Análise da relação entre índices de educação e incidência do Covid-19 no estado do Rio de Janeiro

In [ ]:
#Parte comum:

import pandas as pd
import geopandas as gpd
# from jupyterthemes import jtplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
import basedosdados as bd


states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='bd-geo-covid')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry')

RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

covid = pd.read_csv('caso_full.csv.gz', compression = 'gzip')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int})
covid_rj = covid[(covid["state"] == "RJ") & (covid["date"] == "2021-11-17")].reset_index(drop=True)

covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate"]]

RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')


In [ ]:
fig = px.choropleth(RJ_fulldata,
                   geojson = RJ_fulldata["geometry"],
                   locations = RJ_fulldata.index,
                   color = "cases_per_100k",
                   hover_name = "name",
                   hover_data = ["cases_per_100k", "death_rate"],
                   labels = {
                       "cases_per_100k": "Cases per 100k habitants",
                       "death_rate": "Mortality rate"
                   },
                   color_continuous_scale = "sunsetdark"
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.show()

In [ ]:
#Base de dados IDEB - Educação

ideb = bd.read_sql(
    '''
    SELECT id_municipio AS id, sigla_uf, AVG(ideb) AS ideb , ano
    FROM  basedosdados.br_inep_ideb.municipio
    WHERE ano = 2019 AND rede = 'publica'
    GROUP BY id_municipio, sigla_uf, ano
    ''',
    billing_project_id='bd-geo-covid')


In [ ]:
ideb.head()

In [ ]:
ideb_data =  ideb.astype({"id": int})
ideb_data = ideb_data[ideb_data["id"] > 1000]
ideb_data = ideb_data[(ideb_data["sigla_uf"] == "RJ")].reset_index(drop=True)

RJ_ideb = pd.merge(RJ_data, ideb_data, on='id')

RJ_ideb.head()



In [ ]:
fig = px.choropleth(RJ_ideb,
                   geojson = RJ_ideb["geometry"],
                   locations = RJ_ideb.index,
                   color = "ideb",
                   hover_name = "name",
                   color_continuous_scale = "Blues"
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.show()


In [ ]:
#checar regressao linear, etc.

covid_ideb = pd.merge(RJ_ideb, covid_rj, on='id')

fig = px.scatter(covid_ideb, x="death_rate", y="ideb" , hover_data=['name'])
fig.show()

In [ ]:
#subplots aaaaaaaaaaa

#fig = ps.make_subplots(rows=1, cols=2)
#fig.add_trace( px.scatter(covid_ideb, x="death_rate", y="ideb" , hover_data=['name']))

In [ ]:
idh = pd.read_csv('idh.csv')
RJ_idh = pd.merge(RJ_data, idh, on='name')

RJ_idh.head()

In [ ]:
fig = px.choropleth(RJ_idh,
                   geojson = RJ_idh["geometry"],
                   locations = RJ_idh.index,
                   color = "IDH",
                   hover_name = "name",
                   color_continuous_scale = "Reds"
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.show()


In [ ]:
covid_idh = pd.merge(RJ_idh, covid_rj, on='id')

fig = px.scatter(covid_idh, x="death_rate", y="IDH", hover_data=['name'], marginal_x="histogram")
fig.show()